<a href="https://colab.research.google.com/github/tikendraw/pytorch-explorations/blob/main/pytorch_005_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch import nn
import torch 
import numpy as np
from torchvision import datasets
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor


In [ ]:
import os
os.system('git clone https://github.com/tikendraw/pytorch-explorations.git')
os.chdir('pytorch-explorations')

In [ ]:
from helper_functions import accuracy_fn, print_train_time

In [ ]:
import torchvision as tv

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Get Data

In [ ]:
train_data = datasets.FashionMNIST(root='data', train = True,
                                   transform = tv.transforms.ToTensor(),
                                download = True)

test_data = datasets.FashionMNIST(root='data', train = False,
                                   transform = tv.transforms.ToTensor(),
                                download = True)

100%|██████████| 26421880/26421880 [00:00<00:00, 113301104.95it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 6455046.54it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 57523477.86it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 18236720.43it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
len(train_data), len(test_data)

(60000, 10000)

In [ ]:
train_images , train_labels = train_data.data, train_data.targets
test_images , test_labels = test_data.data, test_data.targets

In [ ]:
# shape check
train_images.shape, train_labels.shape

(torch.Size([60000, 28, 28]), torch.Size([60000]))

In [ ]:
# shape check
test_images.shape , test_labels.shape

(torch.Size([10000, 28, 28]), torch.Size([10000]))

In [ ]:
# to get idx from class
class_to_idx = train_data.class_to_idx
class_to_idx

{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}

In [ ]:
# to get class from idx
idx_to_class = {idx: value for value, idx in class_to_idx.items()}
idx_to_class

{0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot'}

In [ ]:
import random

In [ ]:
def plot_random(n:int=10):
    plt.figure(figsize = (10,30))
    for i in range(n):
        random_image_idx = random.randint(0, len(train_images))
        plt.subplot(n, 5, i+1)
        plt.imshow(train_images[random_image_idx])
        item_name = idx_to_class[train_labels[random_image_idx].item()]
        plt.title(item_name)
    plt.tight_layout()

In [ ]:
# plot_random(n=15)

# DataLoaders

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
BATCH_SIZE = 32

In [ ]:
train_d = DataLoader(train_data, shuffle=True, batch_size =BATCH_SIZE, num_workers=2)
test_d = DataLoader(test_data, shuffle=False, batch_size =BATCH_SIZE, num_workers=2)

In [ ]:
len(train_d), train_d

(1875, <torch.utils.data.dataloader.DataLoader at 0x7f18b7ec8eb0>)

In [ ]:
def plot_data(dataloader, n = 9, seed: int =None):
    
    if seed is not None:
        torch.manual_seed(seed)

    plt.figure(figsize = (10,30))
    for i in range(n):
        image_batch, label_batch = next(iter(dataloader))

        random_image_idx = torch.randint(0, len(image_batch)-1,(1,1)).item()

        image =  image_batch[random_image_idx].squeeze() 
        label = label_batch[random_image_idx].item()

        plt.subplot(n, 5, i+1)
        plt.imshow(image)
        item_name = idx_to_class[label]
        plt.title(item_name)
    plt.tight_layout()

In [ ]:
# plot_data(train_d, n = 15, seed= 44)

# Model

In [ ]:
class FashionModel(nn.Module):
    def __init__(self, input_features: int, output_features: int , units: int =16):
        super().__init__()

        self.layer = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(in_features=input_features, out_features=units),
            # nn.ReLU(),
            nn.Linear(in_features=units, out_features=output_features),
            # nn.ReLU()
        )

    def forward(self, x):
        return self.layer(x)

In [ ]:
model = FashionModel(input_features = 28*28,
                     output_features = 10 , units = 32)

In [ ]:
model = model.to(device)

# Loss , Optimizer and Metrics

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters(), lr = .001)
# accuracy function is imported from helper_function

In [ ]:
from timeit import default_timer as timer
from tqdm.auto import tqdm

# Training Loop

In [ ]:
def train_loop(model: torch.nn.Module,
               data : torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer : torch.optim.Optimizer,
               accuracy_fn ,
               device : torch.device =device):
    
    start = timer()
    train_loss, train_acc = 0, 0

    for x, y in data:
        x, y = x.to(device), y.to(device)

        model.train()

        ypred = model(x)

        optimizer.zero_grad()

        loss = loss_fn(ypred, y)

        train_loss += loss.item()
        train_acc += accuracy_fn(y_true = y, 
                                 y_pred = ypred.argmax(dim = -1))

        loss.backward()

        optimizer.step()

    train_loss /= len(data)
    train_acc  /= len(data)
    end = timer()
    print(f" train loss: {train_loss:.5f}  |  train acc : {train_acc:.5f}%  | time : {end-start:.2f} Sec ", end='\n')
        

    


In [ ]:
def test_loop(model: torch.nn.Module,
               data : torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn ,
               device : torch.device =device):
    
    start = timer()
    test_loss, test_acc = 0, 0

    for x, y in data:
        x, y = x.to(device), y.to(device)

        model.eval()
        
        with torch.inference_mode():
            
            ypred = model(x)

            test_loss += loss_fn(ypred, y).item()
            test_acc += accuracy_fn(y_true = y, 
                                    y_pred = ypred.argmax(dim = -1))


    test_loss /= len(data)
    test_acc  /= len(data)
    end = timer()
    print(f"test loss : {test_loss:.5f}  |  test acc  : {test_acc:.5f}%  | time : {end-start:.2f} Sec ", end='\n')
        


In [ ]:
next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
epochs = 3


for epoch in range(1,epochs+1):

    print(f'Epoch: {epoch:3}/{epochs} | ', end =' ')
    train_loop(model, 
               train_d, 
               loss_fn,
               optimizer,
               accuracy_fn,
               device)
    
    print(' '*16, end =' ')
    test_loop(model, 
               test_d, 
               loss_fn,
               accuracy_fn,
               device)
    # print()

Epoch:   1/3 |   train loss: 1.77320  |  train acc : 53.31667%  | time : 20.59 Sec 
                 test loss : 1.34326  |  test acc  : 63.98762%  | time : 1.85 Sec 
Epoch:   2/3 |   train loss: 1.13466  |  train acc : 66.55500%  | time : 12.57 Sec 
                 test loss : 1.00526  |  test acc  : 66.47364%  | time : 1.86 Sec 
Epoch:   3/3 |   train loss: 0.92023  |  train acc : 69.33333%  | time : 18.25 Sec 
                 test loss : 0.87217  |  test acc  : 69.54872%  | time : 1.87 Sec 


# Evaluation

In [ ]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn, device):
    """Returns a dictionary containing the results of model predicting on data_loader.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            # Make predictions with the model
            y_pred = model(X)
            
            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, 
                                y_pred=y_pred.argmax(dim=1)) # For accuracy, need the prediction labels (logits -> pred_prob -> pred_labels)
        
        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)
        
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}


In [ ]:
# Calculate model 0 results on test dataset
model_results = eval_model(model=model, data_loader=test_d,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn, device=device
)
model_results

{'model_name': 'FashionModel',
 'model_loss': 0.872168779373169,
 'model_acc': 69.54872204472844}

In [ ]:

all_preds = None
all_labels = None

model.eval()
with torch.inference_mode():
    for test_images, test_labels in test_d:
        preds = model(test_images.to(device)).cpu()
        
        if all_preds is None:
            all_preds=preds
            all_labels = test_labels
        else:
            all_preds = torch.cat([all_preds, preds], dim=0)

            all_labels = torch.cat([all_labels, test_labels], dim=0)


In [ ]:
print(all_preds.shape)

torch.Size([10000, 10])


In [ ]:
all_preds_argmax = torch.argmax(all_preds, dim = -1)
all_preds_argmax.shape

torch.Size([10000])

In [ ]:
accuracy = accuracy_fn(y_true=all_labels.cpu(), y_pred = all_preds_argmax.cpu())

In [ ]:
accuracy

69.54

# Model 2

In [ ]:
class ConvModel(nn.Module):
    def __init__(self, channel_dim,  output_classes, hidden_units, kernel_size=3 ):
        super().__init__()

        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels = channel_dim, 
                      out_channels=hidden_units, 
                      kernel_size = kernel_size, 
                      stride = 1, 
                      padding = 1),
            nn.ReLU(),

            nn.Conv2d(in_channels = hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size = kernel_size, 
                      stride = 1, 
                      padding = 1),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size = kernel_size, stride =1, padding=1),

        )

        self.conv_layer2 =  nn.Sequential(
            nn.Conv2d(in_channels = hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size = kernel_size, 
                      stride = 1, 
                      padding = 1),
            nn.ReLU(),

            nn.Conv2d(in_channels = hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size = kernel_size, 
                      stride = 1, 
                      padding = 1),
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size = kernel_size, stride =1, padding=1),

        )

        self.classifier_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=(28*28*hidden_units), out_features = output_classes),
            nn.Softmax(dim=-1)
        )


    def forward(self, x):
        x=  self.conv_layer(x)
        # print('layer 1 shape: ', x.shape)
        
        x=  self.conv_layer2(x)
        # print('layer 2 shape: ', x.shape)

        x= self.classifier_layer(x)
        return x

In [ ]:
model = ConvModel(channel_dim = 1, output_classes = 10, hidden_units = 8).to(device)

In [ ]:
images, lables = next(iter(train_d))

In [ ]:
images = images.to(device)

In [ ]:
images.shape

torch.Size([32, 1, 28, 28])

In [ ]:
out_images = model(images)

In [ ]:
out_images.shape

torch.Size([32, 10])

In [ ]:
# plt.figure(figsize = (20, 30))

# plot_n = 1
# for i in range(8):
#     for j in out_images[i]:
#         plt.subplot(32,8,plot_n)
#         plot_n += 1
#         plt.imshow(j.detach().numpy().squeeze())

# plt.tight_layout()

# Training

In [ ]:
device

'cuda'

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model.parameters(), lr = .05)
# accuracy function is imported from helper_function

In [ ]:
epochs = 20


for epoch in range(1,epochs+1):

    print(f'Epoch: {epoch:3}/{epochs} | ', end =' ')
    train_loop(model, 
               train_d, 
               loss_fn,
               optimizer,
               accuracy_fn,
               device)
    
    print(' '*17, end =' ')
    test_loop(model, 
               test_d, 
               loss_fn,
               accuracy_fn,
               device)
    # print()

Epoch:   1/20 |   train loss: 1.97841  |  train acc : 47.93167%  | time : 15.53 Sec 
                  test loss : 1.91585  |  test acc  : 54.46286%  | time : 2.07 Sec 
Epoch:   2/20 |   train loss: 1.88255  |  train acc : 57.70833%  | time : 15.02 Sec 
                  test loss : 1.81348  |  test acc  : 64.51677%  | time : 2.04 Sec 
Epoch:   3/20 |   train loss: 1.78314  |  train acc : 67.73500%  | time : 16.35 Sec 
                  test loss : 1.77394  |  test acc  : 68.56030%  | time : 2.10 Sec 
Epoch:   4/20 |   train loss: 1.76604  |  train acc : 69.38500%  | time : 15.00 Sec 
                  test loss : 1.76524  |  test acc  : 69.55871%  | time : 2.07 Sec 
Epoch:   5/20 |   train loss: 1.75630  |  train acc : 70.37833%  | time : 15.16 Sec 
                  test loss : 1.75716  |  test acc  : 70.30751%  | time : 3.47 Sec 
Epoch:   6/20 |   train loss: 1.74931  |  train acc : 71.06000%  | time : 15.91 Sec 
                  test loss : 1.75746  |  test acc  : 70.31749%  | tim

In [ ]:
# Calculate model 0 results on test dataset
model_results = eval_model(model=model, data_loader=test_d,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn, device=device
)
model_results

{'model_name': 'ConvModel',
 'model_loss': 1.6383897066116333,
 'model_acc': 82.11861022364218}